In [1]:
import numpy as np
import pandas as pd
import math
from sklearn.metrics import multilabel_confusion_matrix

In [2]:
def lerDatParaPanda(caminhoArquivo):
    df = pd.read_csv(caminhoArquivo, 
                 sep=",", comment='@', header=0)
    return df

In [3]:
class Node:
    def __init__ (self, value):
        self.esquerda = None
        self.data = value
        self.direita = None

In [4]:
class KDtree:
    def constroiKDtree(self, df, profundidade): # Na primeira execucao profundiade = 0
        maxProfundidade = len(df.columns) - 1
        quantidadeNodes = len(df)
        if profundidade > maxProfundidade:
            profundidade = 0
        if (df.columns[profundidade] == 'Class'): #Ignora a coluna "Class", pois a mesma é a que buscamos
            profundidade = profundidade + 1
            if profundidade > maxProfundidade:
                profundidade = 0 
        #Caso base
        if quantidadeNodes == 0:
            return None
        if quantidadeNodes == 1:
            return Node(df)
        else:
            #Media
            if quantidadeNodes % 2 == 0 :
                mediana = int(quantidadeNodes / 2)
            else :
                mediana = int((quantidadeNodes / 2) + 0.5)
            dfEmOrdemDeProfundidade = df.sort_values(df.columns[profundidade])
            Ldata = Node(dfEmOrdemDeProfundidade.iloc[mediana])        
            Ldata.esquerda = self.constroiKDtree(dfEmOrdemDeProfundidade.iloc[:mediana], profundidade + 1)
            Ldata.direita = self.constroiKDtree(dfEmOrdemDeProfundidade.iloc[(mediana+1):],  profundidade + 1)
            return Ldata

In [5]:
class xNN:
    def mainNN(self, df, vizinhos):
        df70 = df.sample(frac = 0.7) # Pega 70% dos dados aleatoriamente
        df30 = df.sample(frac = 0.3) # Pega 30% dos dados aleatoriamente
        tree = KDtree()
        arvoreKd = tree.constroiKDtree(df70, 0)
        proximos = [float('inf')] * vizinhos
        proximosIndex = [None] * vizinhos
        
        df30SemClass = df30.drop(columns=['Class']).to_numpy() # 30% sem a coluna "Class"
        df30SemClassIndex = df30.drop(columns=['Class']).index.to_numpy()
        df30['NovaClass'] = 0
        quantidadeTotalDeTestes = len(df30SemClass)
        indexClass = df70.columns.get_loc('Class')

        quantidadeTestesCorretos = 0
        quantidadeColunas = len(df70.columns)
        for ponto, index in zip(df30SemClass, df30SemClassIndex):
            distancia, Nvizinhos = self.encontraNN(arvoreKd, proximos, proximosIndex, ponto)
            Nvizinhos = np.array(Nvizinhos)

            Nvizinhos = Nvizinhos.reshape(vizinhos, quantidadeColunas)
            classesVerdadeiras = Nvizinhos[:, indexClass] 

            (unique, counts) = np.unique(classesVerdadeiras, return_counts=True)
            frequencies = np.asarray((unique, counts)).T

            rowClassMaxEncontrada = np.argmax(np.max(frequencies, axis=1))
            classeNova = frequencies[rowClassMaxEncontrada][0]
            df30.at[index, 'NovaClass'] = classeNova
        
        analise = analisaResultados();
        analise.mainAnalise(df30)
        return 0

    def encontraNN(self, node, proximos, proximosIndex, ponto):
        
        if (node == None): #Checa se node esta vazio, retorna caso true
            return
        elif (self.checkNodeIsRoot(node) and self.checkDivisorMenorQueProximos(ponto, node, proximos)):
            self.atualizaProximos(ponto, node, proximos, proximosIndex)
            return
        elif(not self.checkNodeIsRoot(node)):
            self.encontraNN(node.esquerda, proximos, proximosIndex, ponto)
            self.encontraNN(node.direita, proximos, proximosIndex, ponto)
        return ([proximos, proximosIndex])

    def checkNodeIsRoot(self, node):
        if (node.esquerda == None and node.direita == None and all(node.data) != None):
            return True
        else:
            return False

    def atualizaProximos(self, ponto, node, proximos, proximosIndex):
        nodeSemClass = node.data.drop(columns=['Class'])
        nodeSeries = nodeSemClass.to_numpy()
        distancia = self.distEuclidiana(ponto, nodeSeries)
        
        max_value = max(proximos)
        max_index = proximos.index(max_value)

        proximos.pop(max_index);
        proximos.insert(max_index, distancia)
        proximosIndex[max_index] = node.data.to_numpy()
        
    def checkDivisorMaiorQueProximos(self, ponto, divisor, proximos):
        divisorSemClass = divisor.data.drop(columns=['Class'])
        divisorSeries = divisorSemClass.to_numpy()
        distancia = self.distEuclidiana(ponto, divisorSeries)
        if (distancia == 0):
            return False
        if (all(distancia >= i  for i in proximos)):
            return True
        else:
            return False

    def checkDivisorMenorQueProximos(self, ponto, divisor, proximos):
        divisorSemClass = divisor.data.drop(columns=['Class'])
        divisorSeries = divisorSemClass.to_numpy()
        distancia = self.distEuclidiana(ponto, divisorSeries)
        if (distancia == 0):
            return False
        if (any(distancia < i for i in proximos)):
            return True
        else:
            return False

    def distEuclidiana(self, num1, num2):
        sum = 0;
        if (len(num1) > 0 and len(num2) > 0):
            num1.shape = len(num1)
            num2.shape = len(num1)
            for i in range(len(num1)):
               d = num1[i] - num2[i]
               sum = sum + d * d
            return math.sqrt(sum)
        else:
            return 0
        
    #Debug
    def mainPrintNotRoots(self, kdTree):
        self.printNotRoots(kdTree)
    def printNotRoots(self, node):
        if (node == None):
            return
        if (self.checkNodeIsRoot(node)):
            print(node.data)
        else:
            self.printNotRoots(node.esquerda)
            self.printNotRoots(node.direita)

In [6]:
class analisaResultados:
    def mainAnalise(self, df):
        totalAnalisado = len(df)
        resultados = df.loc[df['Class'] == df['NovaClass']].groupby('Class').size().sum()

        dfC = df[['Class', 'NovaClass']]
        dfLabels = dfC['Class'].unique()
        res = multilabel_confusion_matrix(dfC[['Class']].to_numpy(), dfC[['NovaClass']].to_numpy(), labels=dfLabels).ravel()
        labelsQtd = len(dfLabels)
        resQtd = len(res)
        if (resQtd > 4):
            newShapeY = int (resQtd / labelsQtd)
            newShapeX = labelsQtd
            res = res.reshape(newShapeX, newShapeY)
            res = res.sum(axis=0)
        (tn, fp, fn, tp) = res
        #Matriz de confusao para encontrar os TP, FP
        print("Precisão: ", self.precisao(tp, fp) , "%")
        print("Revocação: ", self.revocacao(tp, fn), "%")
        print("Acurácia: ", self.acuracia(tp, tn, fp, fn), "%")
        
        return 0
    def acuracia(self, tp, tn, fp, fn):
        result = ((tp + tn) / (tp + tn + fp+ fn)) * 100
        return str(round(result, 2))
    def precisao(self, tp, fp):
        if (tp == fp and tp == 0):
            return 100
        result = (tp / (tp + fp)) * 100
        return str(round(result, 2))
    def revocacao(self, tp, fn):
        result = (tp / (tp + fn)) * 100 
        return str(round(result, 2))

In [7]:
df1 = lerDatParaPanda('data/appendicitis.dat')
df1 = df1.astype(np.float64)
xNNPrincipal = xNN()
result = xNNPrincipal.mainNN(df1, 10)

Precisão:  81.25 %
Revocação:  81.25 %
Acurácia:  81.25 %


In [8]:
df2= lerDatParaPanda('data/banana.dat')
df2 = df2.sample(frac = 0.2)
df2= df2.astype(np.float64)
result = xNNPrincipal.mainNN(df2, 10)

Precisão:  52.2 %
Revocação:  52.2 %
Acurácia:  52.2 %


In [9]:
df3 = lerDatParaPanda('data/bupa.dat')
df3 = df3.astype(np.float64)
result = xNNPrincipal.mainNN(df3, 10)

Precisão:  62.5 %
Revocação:  62.5 %
Acurácia:  62.5 %


In [10]:
df4 = lerDatParaPanda('data/contraceptive.dat')
df4 = df4.sample(frac = 0.2)
df4 = df4.astype(np.float64)
result = xNNPrincipal.mainNN(df4, 10)

Precisão:  35.23 %
Revocação:  35.23 %
Acurácia:  56.82 %


In [11]:
df5 = lerDatParaPanda('data/glass.dat')
df5 = df5.astype(np.float64)
result = xNNPrincipal.mainNN(df5, 10)

Precisão:  18.75 %
Revocação:  18.75 %
Acurácia:  72.92 %


In [12]:
df6 = lerDatParaPanda('data/page-blocks.dat')
df6 = df6.sample(frac = 0.2)
df6 = df6.astype(np.float64)
result = xNNPrincipal.mainNN(df6, 10)

Precisão:  90.24 %
Revocação:  90.24 %
Acurácia:  96.1 %


In [13]:
df7 = lerDatParaPanda('data/phoneme.dat')
df7 = df7.sample(frac = 0.2) #Diminuindo o tamanho dessa base de dados
df7 = df7.astype(np.float64)
result = xNNPrincipal.mainNN(df7, 10)

Precisão:  70.06 %
Revocação:  70.06 %
Acurácia:  70.06 %


In [14]:
df8 = lerDatParaPanda('data/shuttle.dat')
df8 = df8.sample(frac = 0.01) #Diminuindo o tamanho dessa base de dados para +- 500 exemplos
df8 = df8.astype(np.float64)
result = xNNPrincipal.mainNN(df8, 10)

Precisão:  81.03 %
Revocação:  81.03 %
Acurácia:  92.41 %


In [15]:
df9 = lerDatParaPanda('data/tae.dat')
df9 = df9.astype(np.float64)
result = xNNPrincipal.mainNN(df9, 10)

Precisão:  31.11 %
Revocação:  31.11 %
Acurácia:  54.07 %


In [16]:
df10 = lerDatParaPanda('data/titanic.dat')
df10 = df10.sample(frac = 0.2)
df10 = df10.astype(np.float64)
result = xNNPrincipal.mainNN(df10, 10)

Precisão:  67.42 %
Revocação:  67.42 %
Acurácia:  67.42 %
